In [1]:
import pandas as pd
import numpy as nm
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, LongType, IntegerType, FloatType
from pyspark.sql.functions import col, column
from pyspark.sql.functions import expr
from pyspark.sql.functions import split
from pyspark.sql import Row

Clean the data by dropping unnecessary columns (by Suvarana)

In [2]:
raw_df = pd.read_csv("new_dataset.csv", low_memory=False,index_col=0)

In [3]:
raw_df.drop(['Beat','X Coordinate','Y Coordinate','Location','Updated On'],axis=1,inplace=True)

In [4]:
cleaned_df = raw_df.dropna(0,'any')

Encode primary type and location description columns (by Feiting)

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2023-10-26 12:52:44.889223: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
tokenizer = Tokenizer()

In [7]:
tokenizer.fit_on_texts(cleaned_df["Location Description"]) 

In [8]:
tokenizer.fit_on_texts(cleaned_df["Primary Type"]) 

In [9]:
cleaned_df["Loc_Descr_Int"] = tokenizer.texts_to_sequences(cleaned_df["Location Description"])

In [10]:
cleaned_df["Prim_Type_Int"] = tokenizer.texts_to_sequences(cleaned_df["Primary Type"])

In [14]:
encoded_df = cleaned_df.drop(['Location Description','Primary Type'],axis=1)

In [15]:
encoded_df.head

<bound method NDFrame.head of          Case Number                    Date                   Block  IUCR  \
ID                                                                           
12045583    JD226426  05/07/2020 10:24:00 AM     035XX S INDIANA AVE  0820   
12031001    JD209965  04/16/2020 05:00:00 AM         005XX W 32ND ST  0460   
12093529    JD282112  07/01/2020 10:16:00 AM       081XX S COLES AVE  051A   
12178140    JD381597  09/27/2020 11:29:00 PM     065XX S WOLCOTT AVE  0460   
4144897     HL474854  07/10/2005 03:00:00 PM     062XX S ABERDEEN ST  0430   
...              ...                     ...                     ...   ...   
12576294    JE487857  12/23/2021 12:00:00 PM  064XX W IRVING PARK RD  0820   
12551468    JE457970  11/26/2021 12:00:00 AM         008XX E 82ND ST  1310   
12463697    JE347672  08/23/2021 01:45:00 PM      071XX W BELDEN AVE  1570   
12444448    JE327445  08/05/2021 05:40:00 PM     063XX S WESTERN AVE  0860   
12488065    JE380472  09/16/2021 0

In [ ]:
encoded_df.to_csv("preprocessed_data.csv")